<center>

# Web Intelligence

# Who will win the Tennis Australian Open 2020?

## 08 Prediction Simulation

### Riccardo Spolaor (864877)

</center>

In [53]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import python_files.ModelBuilder as modbuild

In [54]:
%store -r

In [55]:
dataframe = pd.read_csv('./datasets/dataframe05.csv', low_memory = False)
modbuild.rename_columns(dataframe)
pred_dataframe = pd.read_csv('./datasets/dataframe_pred.csv', low_memory = False)

## Giocatori

Costruisco la lista dei partecipanti al torneo.

Fonti:
* https://ausopen.com/players#!85991

In [56]:
tournament_players = [
    'ALBOT R.','ANDERSON K.','ANDREOZZI G.','ANDUJAR P.','BASIC M.','BASILASHVILI N.','BAUTISTA AGUT R.','BEDENE A.',
    'BERDYCH T.','BERRETTINI M.','BOLT A.','CARBALLES BAENA R.','CARRENO BUSTA P.','CECCHINATO M.','CHARDY J.',
    'CHUNG H.','CILIC M.','COPIL M.','CORIC B.','CUEVAS P.','DANIEL T.','DARCIS S.','DE MINAUR A.','DELBONIS F.',
    'DIMITROV G.','DJERE L.','DJOKOVIC N.','DONSKOY E.','DUCKWORTH J.','DZUMHUR D.','EBDEN M.','EDMUND K.',
    'EUBANKS C.','EVANS D.','FABBIANO T.','FEDERER R.','FOGNINI F.','FRATANGELO B.','FRITZ T.','FUCSOVICS M.',
    'GARCIA-LOPEZ G.','GARIN C.','GOFFIN D.','GOJOWCZYK P.','GRANOLLERS M.','GULBIS E.','GUNNESWARAN P.','HAASE R.',
    'HARRIS L.','HARRISON R.','HERBERT P.H.','HUMBERT U.','HURKACZ H.','ISNER J.','ISTOMIN D.','ITO T.','IVASHKA I.',
    'JARRY N.','JAZIRI M.','JOHNSON S.','KARLOVIC I.','KECMANOVIC M.','KHACHANOV K.','KLAHN B.','KLIZAN M.',
    'KOHLSCHREIBER P.','KOKKINAKIS T.','KRAJINOVIC F.','KRUEGER M.','KUBLER J.','KUDLA D.','KUKUSHKIN M.',
    'KYRGIOS N.','LAAKSONEN H.','LAJOVIC D.','LI Z.','LOPEZ F.','MAJCHRZAK K.','MANNARINO A.','MARTERER M.','MAYER L.',
    'MCDONALD M.','MEDVEDEV D.','MILLMAN J.','MMOH M.','MOLLEKER R.','MONFILS G.','MUNAR J.','MURRAY A.','NADAL R.',
    'NISHIKORI K.','NISHIOKA Y.','NORRIE C.','OPELKA R.','PAIRE B.','PELLA G.','POLMANS M.','POPYRIN A.','POUILLE L.',
    'QUERREY S.','RAMOS-VINOLAS A.','RAONIC M.','RUBLEV A.','SAKHAROV G.','SANDGREN T.','SCHWARTZMAN D.','SEPPI A.',
    'SHAPOVALOV D.','SIMON G.','SOCK J.','SOUSA P.','SOUSA J.','STRUFF J.L.','THIEM D.','THOMPSON J.','TIAFOE F.',
    'TIPSAREVIC J.','TOMIC B.','TRAVAGLIA S.','TROICKI V.','TSITSIPAS S.','TSONGA J.W.','VANNI L.','VERDASCO F.',
    'VESELY J.','WAWRINKA S.','ZVEREV M.','ZVEREV A.'
]

In [57]:
len(tournament_players)

128

In [58]:
 for p in tournament_players:
        if dataframe[dataframe['PlayerA'] == p].append(dataframe[dataframe['PlayerB'] == p]).shape[0] == 0:
            print (p)

## Random Forest
Costruisco una random forest utilizzando il tuning degli iperparametri elaborato al passo precedente.
Scarto le feature che indicano la stanchezza del giocatore nel torneo, in quanto non conosco i game ed i set dei giocatori durante il torneo:
* ***FadigueTournGamesA>FadigueTournGamesB***
* ***FadigueTournSetsA>FadigueTournSetsB***

In [59]:
rf = RandomForestClassifier(n_estimators = forest_best_features['n_estimators'], 
                            max_depth = forest_best_features['max_depth'], n_jobs = -1)
rf.fit(pred_dataframe.drop(['Winner','FadigueTournGamesA>FadigueTournGamesB',
                    'FadigueTournSetsA>FadigueTournSetsB'], axis = 1), pred_dataframe['Winner'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=175,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

## Funzioni per inseire le Feature dei match

* ***CsvID*** Predo l'ultimo id + 1 e lo applico ad ogni match del torneo;
* ***ATP*** Considero il valore ATP dell'ultimo torneo **Australian Open** 
* ***PtsA>PtsB***: Prendo i punti dell'ultimo match del giocatore e li applico ad ogni match del torneo;
* ***RankA>RankB***: Prendo il rank dell'ultimo match del giocatore e li applico ad ogni match del torneo;
* ***MaxA>MaxB***: Considero la media del massimo odd dei bookmaker del giocatore e la applico ad ogni match del torneo; 
* ***AvgA>AvgB***: Considero la media della media degli odd dei bookmker per ogni giocatore e la applico ad ogni match del torneo;
* ***5_gamesMeanA>5_gamesMeanB***: Considero la media dei game degli ultimi 5 match del giocatore precedenti al torneo e li applico ad ogni match del torneo;
* ***5_setsMeanA>5_setsMeanB***: Considero la media dei set degli ultimi 5 match del giocatore precedenti al torneo e li applico ad ogni match del torneo;
* ***RetiredLast(A/B)***: Considero se il giocatore si è ritirato l'ultimo match;

Le feature rimanenti sono calcolate in base sia allo storico dei match che a quello del torneo.

In [60]:
def get_value(p, value):
    try:
        wdf = dataframe[dataframe['PlayerA'] == p].tail(1)[value + 'A']
        ldf = dataframe[dataframe['PlayerB'] == p].tail(1)[value+'B']
        df = wdf.append(ldf.rename(columns = {value+'B':value+'A'}))
        df.sort_index(inplace= True)
        return df.tail(1).values[0]
    except: 
        if value == 'Pts':
            return dataframe[[value+'A',value+'B']].min(axis = 1).min(axis = 0)
        else:
            return dataframe[[value+'A',value+'B']].max(axis = 1).max(axis = 0)

    
def get_odds(p, value):
    try:
        df = dataframe[dataframe['PlayerA'] == p][value+'A'].append(
            dataframe[dataframe['PlayerB'] == p][value+'B'].rename(
                columns = {value+'A': value+'B'}))
        return df.mean(axis = 0)
    except:
        return dataframe[[value + 'A',value + 'B']].mean(axis = 1).mean(axis = 0)
    
def get_gamesMean(p):
    try:
        df = dataframe[dataframe['PlayerA'] == p].tail(5)[['1A', '2A', '3A', '4A', '5A']].append(
            dataframe[dataframe['PlayerB'] == p].tail(5)[['1B', '2B', '3B', '4B', '5B']].rename(
                columns = {'1B': '1A', '2B':'2A', '3B':'3A', '4B':'4A', '5B':'5A'}))
        df.sort_index(inplace= True, axis = 1)
        return df.tail(5).sum(axis = 1).mean(axis = 0)
    except:
        return 0

def get_setsMean(p):
    try:
        df = dataframe[dataframe['PlayerA'] == p].tail(5)['setsA'].append(
            dataframe[dataframe['PlayerB'] == p].tail(5)['setsB'].rename(
                columns = {'setsB': 'setsA'}))
        df.sort_index(inplace= True)
        return df.tail(5).mean(axis = 0)
    except:
        return 0

def get_played(p):
    wdf = dataframe[dataframe['PlayerA'] == p].tail(1)['PlayedA']
    ldf = dataframe[dataframe['PlayerB'] == p].tail(1)['PlayedB']
    df = wdf.append(ldf.rename(columns = {'PlayedB':'PlayedA'}))
    df.sort_index(inplace= True)
    if(df.empty):
        return 0
    else: 
        return df.tail(1).values[0] + 1
    
def get_won_ratio(p):
    played = get_played(p)
    wdf = dataframe[dataframe['PlayerA'] == p].tail(1)['WonRatioA']
    ldf = dataframe[dataframe['PlayerB'] == p].tail(1)['WonRatioB']
    
    if wdf.empty:
        if ldf.empty:
            return 0
        else:
            return((ldf.values[0]*played))/(played + 1)
    elif ldf.empty:
        return((wdf.values[0]*played) + 1)/(played + 1)  
    elif ldf.index.values > wdf.index.values:
        return((ldf.values[0]*played))/(played + 1)
    else:
        return((wdf.values[0]*played) + 1)/(played + 1)


def get_opponents_won_ratio(pa, pb):
    wdf = dataframe[(dataframe.PlayerA == pa) & (dataframe.PlayerB ==  pb)][['OpponentsWRatioA','OpponentsWRatioB',
                                                                         'OpponentsPlayed']].tail(1)
    ldf = dataframe[(dataframe.PlayerA == pb) & (dataframe.PlayerB ==  pa)][['OpponentsWRatioA','OpponentsWRatioB', 
                                                                          'OpponentsPlayed']].tail(1)
    
    if wdf.empty:
        if ldf.empty:
            return 0
        else:
            return(((ldf['OpponentsWRatioB']*ldf['OpponentsPlayed']))/(ldf['OpponentsPlayed'] + 1)).values[0]
    elif ldf.empty:
        return(((wdf['OpponentsWRatioA']*wdf['OpponentsPlayed']) + 1)/(wdf['OpponentsPlayed'] + 1)).values[0] 
    elif ldf.index.values > wdf.index.values:
        return(((ldf['OpponentsWRatioB']*ldf['OpponentsPlayed']))/(ldf['OpponentsPlayed'] + 1)).values[0]
    else:
        return(((wdf['OpponentsWRatioA']*wdf['OpponentsPlayed']) + 1)/(wdf['OpponentsPlayed'] + 1)).values[0]

def get_retired_last(p):
    wdf = dataframe[dataframe['PlayerA'] == p].tail(1)
    ldf = dataframe[dataframe['PlayerB'] == p].tail(1)
    
    if wdf.empty:
        if ldf.empty:
            return 0
        else:
            return int(ldf['Retired'] == 1)
    elif ldf.empty:
        return 0  
    elif ldf.index.values > wdf.index.values:
        return int(ldf['Retired'] == 1)
    else:
        return 0
    
def get_played_court(p):
    wdf = dataframe[(dataframe.PlayerA == p) & (dataframe.Hard)]['PlayedCourtA'].tail(1)
    ldf = dataframe[(dataframe.PlayerB == p) & (dataframe.Hard)]['PlayedCourtB'].tail(1)
    df = wdf.append(ldf.rename(columns = {'PlayedCourtB':'PlayedCourtA'}))
    df.sort_index(inplace= True)
    if(df.empty):
        return 0
    else: 
        return df.tail(1).values[0] + 1
    
def get_won_court(p):
    played = get_played_court(p)
    wdf = dataframe[(dataframe.PlayerA == p) & (dataframe.Hard)].tail(1)['WonRatioCourtA']
    ldf = dataframe[(dataframe.PlayerB == p) & (dataframe.Hard)].tail(1)['WonRatioCourtB']
    
    if wdf.empty:
        if ldf.empty:
            return 0
        else:
            return((ldf.values[0]*played))/(played + 1)
    elif ldf.empty:
        return((wdf.values[0]*played) + 1)/(played + 1)  
    elif ldf.index.values > wdf.index.values:
        return((ldf.values[0]*played))/(played + 1)
    else:
        return((wdf.values[0]*played) + 1)/(played + 1)
    
def get_hand(p):
    handdf = dataframe[dataframe['PlayerA'] == p].tail(1)['HandA'].append(
        dataframe[dataframe['PlayerB'] == p].tail(1)['HandB'].rename(
            columns = {'HandB': 'HandA'}))
    handdf.sort_index(inplace= True)
    return handdf.tail(1).values[0]

def get_played_vs_same_handed(pa, pb):
    hand= get_hand(pb)
    wdf = dataframe[(dataframe.PlayerA == pa) & (dataframe.HandB ==  hand)]['PlayedVsSameHandedA'].tail(1)
    ldf = dataframe[(dataframe.PlayerB == pa) & (dataframe.HandA ==  hand)]['PlayedVsSameHandedB'].tail(1)
    df = wdf.append(ldf.rename(columns = {'PlayedVsSameHandedB':'PlayedVsSameHandedA'}))
    df.sort_index(inplace= True)
    if(df.empty):
        return 0
    else: 
        return df.tail(1).values[0] + 1
    
def get_won_vs_same_handed(pa, pb):
    hand= get_hand(pb)
    played = get_played_vs_same_handed(pa, pb)
    wdf = dataframe[(dataframe.PlayerA == pa) & (dataframe.HandB ==  hand)]['WonRatioVsSameHandedA'].tail(1)
    ldf = dataframe[(dataframe.PlayerB == pa) & (dataframe.HandA ==  hand)]['WonRatioVsSameHandedB'].tail(1)
    
    if wdf.empty:
        if ldf.empty:
            return 0
        else:
            return((ldf.values[0]*played))/(played + 1)
    elif ldf.empty:
        return((wdf.values[0]*played) + 1)/(played + 1)  
    elif ldf.index.values > wdf.index.values:
        return((ldf.values[0]*played))/(played + 1)
    else:
        return((wdf.values[0]*played) + 1)/(played + 1)

In [61]:
invariate_features = {
    'newcsvID': dataframe['csvID'].max() + 1,
    'rank': {p:get_value(p, 'Rank') for p in tournament_players},
    'max': {p:get_odds(p, 'Max') for p in tournament_players},
    'avg': {p:get_odds(p, 'Avg') for p in tournament_players},
    'pts': {p:get_value(p, 'Pts') for p in tournament_players},
    '5_games_mean': {p:get_gamesMean(p) for p in tournament_players},
    '5_sets_mean': {p:get_setsMean(p) for p in tournament_players},
    'retired': {p:get_retired_last(p) for p in tournament_players},
    'hand': {p:get_hand(p) for p in tournament_players}
}

## Predizione
Predico ogni match costruendo una nuova riga del dataframe inserendo le feature calcolate come indicato sopra.

1. La lista dei giocatori viene inizialmente mescolata;
* I match sono simulati scorrendo la lista a coppie di giocatori;
* Elimino dalla lista dei giocatori il perdente di ogni match;
* Una volta analizzata tutta la lista ripeto il procedimento dal passo 2 fino a che non ottengo un solo giocatore che corrisponde al vincitore del torneo.

In [62]:
def get_basic_row(pa,pb,n_round):
    df = pd.DataFrame(data = {
                    'csvID': [invariate_features['newcsvID']], 
                    'ATP': [6], 
                    'Location': [ordered_locations['Melbourne']], 
                    'Tournament': [tournamentsDict['Australian Open']], 
                    'Series': [4], 
                    'Court': [1], 
                    'Round': [n_round],
                    'Best of': [5], 
                    'PtsFilledA': [1], 
                    'PtsFilledB': [1], 
                    'MaxFilledA': [1], 
                    'MaxFilledB': [1],
                    'AvgFilledA': [1], 
                    'AvgFilledB': [1], 
                    'Carpet': [0], 
                    'Clay': [0], 
                    'Grass': [0], 
                    'Hard': [1],
                    'FEDERER R.B': [int(pb=='FEDERER R.')], 
                    'FERRER D.A': [int(pa=='FERRER D.')], 
                    'FERRER D.B': [int(pb=='FERRER D.')],
                    'ROBREDO T.A': [int(pa=='ROBREDO T.')],
                    'ROBREDO T.B': [int(pb=='ROBREDO T.')],
                    'NALBANDIAN D.B': [int(pb=='NALBANDIAN D.')], 
                    'DAVYDENKO N.A': [int(pa=='DAVYDENKO N.')],
                    'DAVYDENKO N.B': [int(pb=='DAVYDENKO N.')],
                    'OtherA': [int(pa not in ['FERRER D.',
                                           'ROBREDO T.',
                                           'DAVYDENKO N.'])], 
                    'OtherB': [int(pb not in ['FEDERER R.',
                                            'FERRER D.'
                                            'ROBREDO T.',
                                            'NALBANDIAN D.',
                                            'DAVYDENKO N.'])],
                    'RankA': [invariate_features['rank'][pa]],
                    'RankB': [invariate_features['rank'][pb]],
                    'MaxA': [invariate_features['max'][pa]],
                    'MaxB': [invariate_features['max'][pb]],
                    'AvgA': [invariate_features['avg'][pa]],
                    'AvgB': [invariate_features['avg'][pb]],
                    'PtsA':  [invariate_features['pts'][pa]],
                    'PtsB':  [invariate_features['pts'][pb]],
                    '5_gamesMeanA': [invariate_features['5_games_mean'][pa]],
                    '5_gamesMeanB': [invariate_features['5_games_mean'][pb]],
                    '5_setsMeanA': [invariate_features['5_sets_mean'][pa]],
                    '5_setsMeanB': [invariate_features['5_sets_mean'][pb]],
                    'PlayerA': [pa],
                    'PlayerB': [pb],
                    'HandA': [invariate_features['hand'][pa]],
                    'HandB': [invariate_features['hand'][pb]],
                    'PlayedA': [get_played(pa)],
                    'PlayedB': [get_played(pb)],
                    'WonRatioA': [get_won_ratio(pa)],
                    'WonRatioB': [get_won_ratio(pb)],
                    'OpponentsWRatioA': [get_opponents_won_ratio(pa, pb)],
                    'OpponentsWRatioB': [get_opponents_won_ratio(pb, pa)],
                    'RetiredLastA': [invariate_features['retired'][pa]],
                    'RetiredLastB': [invariate_features['retired'][pb]],
                    'PlayedCourtA': [get_played_court(pa)],
                    'PlayedCourtB': [get_played_court(pb)],
                    'WonRatioCourtA': [get_won_court(pa)],
                    'WonRatioCourtB': [get_won_court(pb)],
                    'PlayedVsSameHandedA': [get_played_vs_same_handed(pa, pb)],
                    'PlayedVsSameHandedB': [get_played_vs_same_handed(pb, pa)],
                    'WonRatioVsSameHandedA': [get_won_vs_same_handed(pa, pb)],
                    'WonRatioVsSameHandedB': [get_won_vs_same_handed(pb, pa)],
               })
    if n_round > 0:
        df['RetiredLastA'] = 0
        df['RetiredLastB'] = 0
    
    df['RankA>RankB'] = df.apply(lambda x: int(x['RankA'] > x['RankB']), axis=1)
    df['AvgA>AvgB'] = df.apply(lambda x: int(x['AvgA'] > x['AvgB']), axis=1)
    df['MaxA>MaxB'] = df.apply(lambda x: int(x['MaxA'] > x['MaxB']), axis=1)
    df['PtsA>PtsB'] = df.apply(lambda x : int(x['PtsA'] > x['PtsB']), axis = 1)

    df['OpponentsWRatioA>OpponentsWRatioB'] = df.apply(
        lambda x: int(x['OpponentsWRatioA'] > x['OpponentsWRatioB']), axis=1)

    df['PlayedA>PlayedB'] = df.apply(lambda x: int(x['PlayedA'] > x['PlayedB']), axis=1)
    df['WonRatioA>WonRatioB'] = df.apply(lambda x: int(x['WonRatioA'] > x['WonRatioB']), axis=1)

    df['PlayedCourtA>PlayedCourtB'] = df.apply(lambda x: int(x['PlayedCourtA'] > x['PlayedCourtB']),
                                                             axis=1)
    df['WonRatioCourtA>WonRatioCourtB'] = df.apply(
        lambda x: int(x['WonRatioCourtA'] > x['WonRatioCourtB']), axis=1)

    df['PlayedVsSameHandedA>PlayedVsSameHandedB'] = df.apply(
        lambda x: int(x['PlayedVsSameHandedA'] > x['PlayedVsSameHandedB']), axis=1)
    df['WonRatioVsSameHandedA>WonRatioVsSameHandedB'] = df.apply(
        lambda x: int(x['WonRatioVsSameHandedA'] > x['WonRatioVsSameHandedB']), axis=1)

    df['5_gamesMeanA>5_gamesMeanB'] = df.apply(
        lambda x: int(x['5_gamesMeanA'] > x['5_gamesMeanB']), axis=1)
    df['5_setsMeanA>5_setsMeanB'] = df.apply(
        lambda x: int(x['5_setsMeanA'] > x['5_setsMeanB']), axis=1)
    return df

In [63]:
def predict_result(row):
    winner = rf.predict(row.drop(['WonRatioCourtA', 'WonRatioCourtB', 'WonRatioA', 'WonRatioB', 
                      'OpponentsWRatioA','OpponentsWRatioB', 'RankB', 'RankA',  'PtsA',
                      'PtsB', 'MaxA', 'MaxB', 'AvgA', 'AvgB',  'PlayedA', 'PlayedB','PlayedCourtA', 
                      'PlayedCourtB', 'PlayedVsSameHandedA', 'PlayedVsSameHandedB','WonRatioVsSameHandedA',
                      'WonRatioVsSameHandedB', '5_gamesMeanA', '5_gamesMeanB', '5_setsMeanA', '5_setsMeanB', 'PlayerA',
                      'PlayerB', 'HandA', 'HandB'], axis=1))
    return winner[0]

In [64]:
def get_round_prediction(player_list, n_round):
    
    new_list = []
    round_dict = {0: '1st Round', 1: '2nd Round', 2: '3rd Round', 3: '4th Round', 5: 'Quarterfinals', 6:'Semifinals', 
                  7:'The Final'}
    global dataframe 
    
    for i,d in enumerate(zip(player_list[0::2], player_list[1::2])):
        pa,pb = d
        row = get_basic_row(pa, pb, n_round)
        winner = predict_result(row)
        
        if winner:
            print(round_dict[n_round],'- Partita:', i, '- Vincitore:', pb, '- Perdente:', pa)
    
            col_to_invert = ['Player', 'Rank', 'Pts', 'PtsFilled', 'Max', 'Avg', 'MaxFilled', 'AvgFilled',  
                                   'Played', 'WonRatio','WonRatioCourt', 'PlayedCourt', 'OpponentsWRatio',  
                                   'RetiredLast','5_gamesMean', '5_setsMean',  'PlayedVsSameHanded',
                                   'WonRatioVsSameHanded', 'Hand']
    
            for f in col_to_invert:
                row.rename(columns = {f+'A': f+'B', f+'B': f + 'A'}, inplace= True)
            
            new_list += [pb]
            dataframe = dataframe.append(row, ignore_index = True, sort = False)
        else:
            print(round_dict[n_round],'- Partita:', i, '- Vincitore:', pa, '- Perdente:', pb)
            new_list += [pa]
            dataframe = dataframe.append(row, ignore_index = True, sort = False)
        
    print('\n')
        
    return new_list

In [65]:
from random import shuffle
shuffle(tournament_players)
tournament_players = get_round_prediction(tournament_players, 0)
tournament_players = get_round_prediction(tournament_players, 1)
tournament_players = get_round_prediction(tournament_players, 2)
tournament_players = get_round_prediction(tournament_players, 3)
tournament_players = get_round_prediction(tournament_players, 5)
tournament_players = get_round_prediction(tournament_players, 6)
print('Vincitore:', tournament_players[0])

1st Round - Partita: 0 - Vincitore: KRAJINOVIC F. - Perdente: MILLMAN J.
1st Round - Partita: 1 - Vincitore: BERDYCH T. - Perdente: BAUTISTA AGUT R.
1st Round - Partita: 2 - Vincitore: WAWRINKA S. - Perdente: BEDENE A.
1st Round - Partita: 3 - Vincitore: ZVEREV A. - Perdente: MURRAY A.
1st Round - Partita: 4 - Vincitore: ANDERSON K. - Perdente: VERDASCO F.
1st Round - Partita: 5 - Vincitore: STRUFF J.L. - Perdente: MARTERER M.
1st Round - Partita: 6 - Vincitore: THIEM D. - Perdente: KUBLER J.
1st Round - Partita: 7 - Vincitore: DONSKOY E. - Perdente: MMOH M.
1st Round - Partita: 8 - Vincitore: HUMBERT U. - Perdente: KUDLA D.
1st Round - Partita: 9 - Vincitore: CORIC B. - Perdente: MCDONALD M.
1st Round - Partita: 10 - Vincitore: FABBIANO T. - Perdente: EDMUND K.
1st Round - Partita: 11 - Vincitore: THOMPSON J. - Perdente: CECCHINATO M.
1st Round - Partita: 12 - Vincitore: BERRETTINI M. - Perdente: MOLLEKER R.
1st Round - Partita: 13 - Vincitore: KUKUSHKIN M. - Perdente: MEDVEDEV D.
1st

4th Round - Partita: 3 - Vincitore: POLMANS M. - Perdente: ANDREOZZI G.
4th Round - Partita: 4 - Vincitore: DJOKOVIC N. - Perdente: PELLA G.
4th Round - Partita: 5 - Vincitore: FOGNINI F. - Perdente: RAONIC M.
4th Round - Partita: 6 - Vincitore: CARRENO BUSTA P. - Perdente: DANIEL T.
4th Round - Partita: 7 - Vincitore: GOFFIN D. - Perdente: CHARDY J.


Quarterfinals - Partita: 0 - Vincitore: KRAJINOVIC F. - Perdente: HUMBERT U.
Quarterfinals - Partita: 1 - Vincitore: GRANOLLERS M. - Perdente: POLMANS M.
Quarterfinals - Partita: 2 - Vincitore: DJOKOVIC N. - Perdente: FOGNINI F.
Quarterfinals - Partita: 3 - Vincitore: CARRENO BUSTA P. - Perdente: GOFFIN D.


Semifinals - Partita: 0 - Vincitore: KRAJINOVIC F. - Perdente: GRANOLLERS M.
Semifinals - Partita: 1 - Vincitore: DJOKOVIC N. - Perdente: CARRENO BUSTA P.


Vincitore: KRAJINOVIC F.


In [69]:
with pd.option_context('display.max_columns',None):
    display(dataframe.tail(5))

,Winner,csvID,ATP,Location,Tournament,Date,Series,Court,Round,Best of,PlayerA,PlayerB,PlayedA,PlayedB,WonRatioA,WonRatioB,RankA,RankFilledA,RankB,RankFilledB,1A,1B,2A,2B,3A,3B,4A,4B,5A,5B,setsA,setsB,FadigueTournGamesA,FadigueTournGamesB,FadigueTournSetsA,FadigueTournSetsB,PtsA,PtsFilledA,PtsB,PtsFilledB,MaxA,MaxFilledA,MaxB,PlayedCourtA,PlayedCourtB,WonRatioCourtA,WonRatioCourtB,MaxFilledB,AvgA,AvgFilledA,AvgB,AvgFilledB,Carpet,Clay,Grass,Hard,Awarded,Completed,Disqualified,Retired,Sched,Walkover,FEDERER R.A,FEDERER R.B,NADAL R.A,NADAL R.B,DJOKOVIC N.A,DJOKOVIC N.B,MURRAY A.A,MURRAY A.B,RODDICK A.A,RODDICK A.B,FERRER D.A,FERRER D.B,BERDYCH T.A,BERDYCH T.B,HEWITT L.A,HEWITT L.B,DEL POTRO J. M.A,DEL POTRO J. M.B,TSONGA J.W.A,TSONGA J.W.B,CILIC M.A,CILIC M.B,WAWRINKA S.A,WAWRINKA S.B,GASQUET R.A,GASQUET R.B,NISHIKORI K.A,NISHIKORI K.B,MONFILS G.A,MONFILS G.B,FERRERO J.C.A,FERRERO J.C.B,ROBREDO T.A,ROBREDO T.B,NALBANDIAN D.A,NALBANDIAN D.B,RAONIC M.A,RAONIC M.B,AGASSI A.A,AGASSI A.B,HAAS T.A,HAAS T.B,ISNER J.A,ISNER J.B,MOYA C.A,MOYA C.B,DAVYDENKO N.A,DAVYDENKO N.B,GONZALEZ F.A,GONZALEZ F.B,OtherA,OtherB,RankA>RankB,MaxA>MaxB,AvgA>AvgB,PtsA>PtsB,OpponentsPlayed,OpponentsWRatioA,OpponentsWRatioB,OpponentsWRatioA>OpponentsWRatioB,FadigueTournGamesA>FadigueTournGamesB,FadigueTournSetsA>FadigueTournSetsB,WalkoverLastA,WalkoverLastB,RetiredLastA,RetiredLastB,PlayedA>PlayedB,WonRatioA>WonRatioB,PlayedCourtA>PlayedCourtB,WonRatioCourtA>WonRatioCourtB,5_gamesMeanA,5_setsMeanA,5_gamesMeanB,5_setsMeanB,5_gamesMeanA>5_gamesMeanB,5_setsMeanA>5_setsMeanB,HandA,HandB,PlayedVsSameHandedA,PlayedVsSameHandedB,WonRatioVsSameHandedA,WonRatioVsSameHandedB,PlayedVsSameHandedA>PlayedVsSameHandedB,WonRatioVsSameHandedA>WonRatioVsSameHandedB
52066,NaN,19,6.0,107,22,NaN,4,1,5,5.0,GRANOLLERS M.,POLMANS M.,450,9,0.455547,0.525000,100.0,NaN,172.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,551.0,1,301.0,1,3.718985,1,3.806000,200,8,0.424952,0.592593,1,3.057346,1,3.354000,1,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,1,1,0,0,0,1,NaN,0.0,0.0,0,NaN,NaN,NaN,NaN,0,0,1,0,1,0,14.2,1.0,13.4,1.2,1,0,0,0,390,8,0.420504,0.592593,1,0
52067,NaN,19,6.0,107,22,NaN,4,1,5,5.0,DJOKOVIC N.,FOGNINI F.,1042,635,0.831094,0.535430,2.0,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8945.0,1,2370.0,1,1.447203,1,3.209098,662,263,0.844411,0.482866,1,1.372959,1,2.662819,1,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,1,1,0,0,0,1,NaN,1.0,0.0,1,NaN,NaN,NaN,NaN,0,0,1,1,1,1,11.8,1.4,7.8,0.6,1,1,0,0,893,532,0.842106,0.537589,1,1
52068,NaN,19,6.0,107,22,NaN,4,1,5,5.0,CARRENO BUSTA P.,GOFFIN D.,326,411,0.539865,0.598534,34.0,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1330.0,1,2325.0,1,2.733199,1,2.594767,162,264,0.567856,0.602258,1,2.396553,1,2.259484,1,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,1,1,1,1,1,0,NaN,0.5,0.5,0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,8.4,1.4,9.4,0.6,0,1,0,0,274,342,0.525542,0.584786,0,0
52069,NaN,19,6.0,107,22,NaN,4,1,6,5.0,KRAJINOVIC F.,GRANOLLERS M.,127,451,0.503809,0.456752,46.0,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1068.0,1,551.0,1,3.538033,1,3.718985,93,201,0.515896,0.427799,1,3.006230,1,3.057346,1,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,1,1,0,0,0,1,NaN,0.0,1.0,0,NaN,Na